### Setting

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
# LoRA 파일 경로 및 파일명 설정
lora_name = "Pastel_Dreamscape"
gdrive_lora_dir = f"/content/drive/MyDrive/Loras/{lora_name}/output"  # LoRA 폴더 경로
lora_weight_file = "Pastel_Dreamscape-10.safetensors"                 # LoRA 파일명
adapter_name = "default"                                  # 적용될 어댑터 이름 (아래 셀에서 자동 세팅)


In [9]:
from safetensors.torch import safe_open

# LoRA safetensor 파일 경로
lora_path = f"/content/drive/MyDrive/Loras/{lora_name}/output/{lora_weight_file}"

# 메타데이터 확인
with safe_open(lora_path, framework="pt") as f:
    metadata = f.metadata()  # ← () 붙여줘야 함
    print("✅ 메타데이터:", metadata)

    # 어댑터 이름 추출 (없으면 default로)
    adapter_name = metadata.get("ss_adapter_name", "default")
    print("👉 어댑터 이름:", adapter_name)


✅ 메타데이터: {'modelspec.resolution': '1024x1024', 'ss_mixed_precision': 'fp16', 'ss_training_comment': 'None', 'ss_loss_type': 'l2', 'ss_v2': 'False', 'ss_ip_noise_gamma_random_strength': 'False', 'ss_new_sd_model_hash': '31e35c80fc4829d14f90153f4c74cd59c90b779f6afe05a74cd6120b893f7e5b', 'modelspec.architecture': 'stable-diffusion-xl-v1-base/lora', 'ss_num_train_images': '250', 'ss_dataset_dirs': '{"dataset": {"n_repeats": 10, "img_count": 25}}', 'ss_fp8_base': 'False', 'ss_tag_frequency': '{"dataset": {"a group of wind turbines in a field": 1, "a sunset over a beach with boats in the water": 1, "a sunset view of a beach with a boat in the water": 1, "a person walking on a beach with a surfboard": 1, "a boat is sitting on the frozen water": 1, "a cow standing in a field at sunset": 1, "a lighthouse on a cliff with a beautiful sunset": 1, "a beach with a pink sky and a few people": 1, "a person is standing on a beach with a surfboard": 1, "a couple of people on a beach with a surfboard": 1

### Generate Function

In [10]:
from diffusers import AutoPipelineForText2Image, StableDiffusionPipeline, StableDiffusion3Pipeline
from datetime import datetime
import torch
import gc
from PIL import Image
from IPython.display import display

default_image_size = {
    "sd15": (512, 512),
    "sdxl": (1024, 1024),
    "sd35m": (1024, 1024),
}

model_dict = {
    "sd15": "runwayml/stable-diffusion-v1-5",
    "sdxl": "stabilityai/stable-diffusion-xl-base-1.0",
    "sd35m": "stabilityai/stable-diffusion-3.5-medium",
}

def print_gpu_usage(stage=""):
    allocated = torch.cuda.memory_allocated() / 1e9
    reserved = torch.cuda.memory_reserved() / 1e9
    print(f"[GPU: {stage}] Allocated: {allocated:.2f} GB | Reserved: {reserved:.2f} GB")

def apply_lora(pipeline, lora_path, weight_file, adapter_name="default", strength=1.0):
    pipeline.load_lora_weights(
        lora_path,
        weight_name=weight_file,
        adapter_name=adapter_name
    )
    pipeline.set_adapters([adapter_name], adapter_weights=[strength])


In [11]:
def generate_image(
    model_version="sdxl",
    lora_path=None,
    lora_weight=None,
    adapter_name="style",
    strength=1.0,
    positive_prompt="a beautiful illustration",
    negative_prompt="blurry, low quality",
    image_size=None,
    save_dir="/content"
):
    print(f"\n=== Stable Diffusion [{model_version}] 시작 ===")

    if image_size is None:
        image_size = default_image_size.get(model_version.lower(), (512, 512))

    model_id = model_dict.get(model_version.lower())
    if model_id is None:
        raise ValueError("지원되지 않는 모델입니다.")

    if model_version == "sd35m":
        pipeline = StableDiffusion3Pipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            use_safetensors=True
        ).to("cuda")
    elif model_version in ["sd15"]:
        pipeline = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            use_safetensors=True
        ).to("cuda")
    else:  # sdxl
        pipeline = AutoPipelineForText2Image.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            variant="fp16" if "xl" in model_version else None,
            use_safetensors=True
        ).to("cuda")

    print_gpu_usage("Base 모델 로드 후")

    if lora_path and lora_weight:
        print(f"[INFO] LoRA 적용 중: {lora_weight}")
        apply_lora(pipeline, lora_path, lora_weight, adapter_name=adapter_name, strength=strength)

    print(f"[INFO] 이미지 생성 중...")
    with torch.no_grad():
        image = pipeline(
            prompt=positive_prompt,
            negative_prompt=negative_prompt,
            height=image_size[1],
            width=image_size[0]
        ).images[0]
    print_gpu_usage("이미지 생성 후")

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{save_dir}/gen_{model_version}_{adapter_name}_{timestamp}.png"
    image.save(filename)
    print(f"[INFO] 이미지 저장됨: {filename}")
    display(image)

    del image
    del pipeline
    gc.collect()
    torch.cuda.empty_cache()
    print("[INFO] 메모리 정리 완료 ✅\n")

    return filename


# 🔬 A/B Test: LoRA 적용 여부 비교

동일한 프롬프트를 기준으로 다음 두 케이스를 비교합니다:
- **A (Base Only)**: LoRA 미적용, SDXL만 사용
- **B (With LoRA)**: Gogh_Lora 적용

이미지 스타일의 변화, 색감, 붓터치 등의 변화를 관찰하세요.

In [26]:
image_save_path = '/content'

In [27]:
positive_prompt = (
    "A dreamy night street scene with soft glowing streetlights, "
    "a couple walking hand in hand under pastel-colored trees, "
    "lavender and pink moonlight reflecting on wet pavement, "
    "gentle fog drifting through the street, "
    "quiet romantic atmosphere, watercolor painting style, "
    "emotional lighting, cinematic composition, tranquil mood"
)

negative_prompt = (
    "blurry, distorted faces, harsh shadows, cartoon style, oversaturated, crowded background, anime"
)


In [28]:
print("🅰️ Generating image WITHOUT LoRA (Base Only)")

generate_image(
    model_version="sdxl",
    lora_path=None,
    lora_weight=None,
    adapter_name=None,
    strength=0,
    positive_prompt=positive_prompt,
    negative_prompt=negative_prompt,
    save_dir=image_save_path
)

Output hidden; open in https://colab.research.google.com to view.

In [29]:
print(f"🅱️ Generating image WITH {lora_name}")

generate_image(
    model_version="sdxl",
    lora_path=f"/content/drive/MyDrive/Loras/{lora_name}/output",
    lora_weight=lora_weight_file,
    adapter_name=adapter_name,
    strength=1.0,
    positive_prompt=positive_prompt,
    negative_prompt=negative_prompt,
    save_dir=image_save_path
)

Output hidden; open in https://colab.research.google.com to view.